In [27]:
import numpy as np
from mnist import MNIST
mndata = MNIST('data/')
train = mndata.load_training()
test = mndata.load_testing()

## Format Data

In [33]:
def format_data(data):
    """
    Each example in the MINST dataset is 28x28 pixels 
    and hence contains 784 pixels i.e. features or input variables.
    
    Each dataset is a tuple (x,y) where 0th element x is a
    list of inputs for each example.
    The 1st element y is the label that tells whats the actual 
    digit that the image corresponds to.
    
    With the above in mind, lets write our data formatter to get the desired format.
    Since we are doing 10 class classification, so as part of formatting 
    we will also need to convert he single output into an array of 10 outputs, with 
    the element at the label place as 1 and rest elements as 0
    """
    return zip([np.reshape(items, (784, 1)) for items in data[0]],
                          [create_multiclass_output(label) for label in data[1]])
    
    
                          
def create_multiclass_output(label):
    multiclass_output = np.zeros((10,1))
    multiclass_output[label] = 1.0
    return multiclass_output
    
train_formatted = format_data(train)
test_formatted = format_data(test)

## Neural Network Class

In [122]:
import random
import numpy as np

class NeuralNet:
    
    def __init__(self,design):
        
        """ we will initiate the layer here with 
        layer size and number of neurons in each layer 
        as input random weights and biases. 
        
        The first layer is an input layer and hence 
        we would not set any biases for it.
        
        Remember, we need weights for every connection 
        possible between neurons of adjoining layers. 
        """
        
        self.layers = len(design)
        self.design = design
        self.biases = [np.random.randn(y, 1) for y in design[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(design[:-1], design[1:])]
        
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = self.sigmoid(np.dot(w, a)+b)
        return a
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_weights_biases(mini_batch, eta)
            if test_data:
                print "Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test)
            else:
                print "Epoch {0} complete".format(j)
            

    def update_weights_biases(self, dataset, eta):
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in dataset:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(dataset))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(dataset))*nb for b, nb in zip(self.biases, nabla_b)]
    
    def backprop(self, x, y):
        
        """
        In backpropagation, first we feed forward, 
        and calculate the cost function. 
        Then we adjust the weights and biases in a way that 
        the value of cost function could be minimised
        """
            
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # feedforward
        activation = x
        list_activation = [x] 
        list_z = []
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            list_z.append(z)
            activation = self.sigmoid(z)
            list_activation.append(activation)
        
        # backward pass
        
        # implementing equation 1 of backpropagation
        delta = (list_activation[-1]-y) * self.sigmoid_derivative(list_z[-1])
        
        # implementing equation 3 of backpropagation
        nabla_b[-1] = delta
        
        # implementing equation 4 of backpropagation
        nabla_w[-1] = np.dot(delta, list_activation[-2].transpose())
        
        for l in xrange(2, self.layers):
            z = list_z[-l]
            delta = np.dot(self.weights[-l+1].transpose(), delta) * self.sigmoid_derivative(z)
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, list_activation[-l-1].transpose())
        return (nabla_b, nabla_w)
    
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        a = 0
        for (x, y) in test_results:
            multiclass_output = create_multiclass_output(x)
            if (y==multiclass_output).all():
                a = a+1
        return a
         
    def sigmoid(self,z):
        return 1.0/(1.0+np.exp(-z))

    def sigmoid_derivative(self,z):
        return self.sigmoid(z)*(1-self.sigmoid(z))

## Start Training

In [128]:
net = NeuralNet([784, 100,200, 10])
net.SGD(train_formatted, 30, 10, 0.9, test_data = test_formatted)

/Library/Python/2.7/site-packages/ipykernel_launcher.py:106: RuntimeWarning: overflow encountered in exp


Epoch 0: 2683 / 10000
Epoch 1: 4610 / 10000
Epoch 2: 4279 / 10000
Epoch 3: 4117 / 10000
Epoch 4: 3192 / 10000
Epoch 5: 5053 / 10000
Epoch 6: 5552 / 10000
Epoch 7: 4851 / 10000
Epoch 8: 5358 / 10000
Epoch 9: 5735 / 10000
Epoch 10: 6100 / 10000
Epoch 11: 5867 / 10000
Epoch 12: 5635 / 10000
Epoch 13: 6261 / 10000
Epoch 14: 5760 / 10000
Epoch 15: 6371 / 10000
Epoch 16: 5531 / 10000
Epoch 17: 5921 / 10000
Epoch 18: 5748 / 10000
Epoch 19: 5466 / 10000
Epoch 20: 5502 / 10000
Epoch 21: 5843 / 10000
Epoch 22: 6190 / 10000
Epoch 23: 5491 / 10000
Epoch 24: 6247 / 10000
Epoch 25: 5471 / 10000
Epoch 26: 5752 / 10000
Epoch 27: 5816 / 10000
Epoch 28: 5917 / 10000
Epoch 29: 5880 / 10000
